# Cleanum of the collected Data and Feature finding (Fuel and Source/Destination)

# NOTE: The code does not work in a single run, since one of the API servers is very slow and data had to be run in a couple of batches thouought several days, with intermediate code corrections


In [80]:
import pandas as pd
import requests
import time
import json
import numpy as np
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [81]:
#Preliminary Parameters
file_name = 'completed_fligths_last.csv'
pd.set_option('display.max_rows', None)
req_window = 2
min_threashold = 150

In [82]:
def find_src_airport_and_country(lon, lat):

    print("req made")
    url = "https://aviation-reference-data.p.rapidapi.com/airports/search"

    querystring = {"lat": f"{lat}","lon": f"{lon}","radius":"10"}

    headers = {
    	"X-RapidAPI-Key": "1daecde3ebmshc01fb168d310d25p1e0b29jsn22e5a1c9059e",
    	"X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        data = response.json()
        if data:
            first_object = data[0]
            iata_code = first_object.get("iataCode")
            alpha2_country_code = first_object.get("alpha2countryCode")
            return iata_code, alpha2_country_code
    return "NO DATA", "NO DATA"

In [83]:
def find_dest_airport_and_country(lon, lat):

    print("req made")
    url = "https://aviation-reference-data.p.rapidapi.com/airports/search"
    
    querystring = {"lat": f"{lat}","lon": f"{lon}","radius":"10"}

    headers = {
    	"X-RapidAPI-Key": "1daecde3ebmshc01fb168d310d25p1e0b29jsn22e5a1c9059e",
    	"X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        data = response.json()
        if data:
            first_object = data[0]
            iata_code = first_object.get("iataCode")
            alpha2_country_code = first_object.get("alpha2countryCode")
            return iata_code, alpha2_country_code
    return "NO DATA", "NO DATA"

In [84]:
def get_fuel_consumption(df):

    #df = df.head(200)
    icaos = df['icao'].tolist()
    distances = df['distance(km)'].tolist()

    icaos = [str(item) for item in icaos]
    distances = [str(item) for item in distances]

    for i in range(len(icaos)):
        # Remove commas from the current string item
        icaos[i] = icaos[i].replace(",", "")

    icaos_string = '[' + ','.join(icaos) + ']'
    distances_string = '[' + ','.join(distances) + ']'

    #print(icaos_string)
    #print(distances_string)

    url = f"https://despouy.ca/flight-fuel-api/q/?aircraft=[{icaos_string}]&distance=[{distances_string}]"

    response = requests.get(url)
    data = response.json()

    features_list = []
    for item in data:
        icao = item.get('icao24', 'NO DATA')
        model = item.get('model', 'NO DATA')
        co2 = item.get('co2', 'NO DATA')
        fuel = item.get('fuel', 'NO DATA')
        features_list.append({'icao': icao, 'model': model, 'co2': co2, 'fuel': fuel})
    
    # Convert list of dictionaries to DataFrame
    df_fuel = pd.DataFrame(features_list)

    merged_df = pd.merge(df, df_fuel, on='icao', how='left')
    merged_df = merged_df.fillna('NO DATA')

    return merged_df

    #df.to_csv("fuel.csv", index=False)

In [85]:
def pretty_df_func(df):

    #columns_to_keep = ['icao', 'model', 'co2', 'fuel', 'distance(km)', 'flight_duration', 'src_country', 'dest_country', 'src_airport', 'dest_airport']
    #df_pretty = df.loc[:, columns_to_keep]

    #df_pretty.to_csv("pretty.csv", index=False)
    df.to_csv("completed_second_half.csv", index=False)

In [86]:
def predict_distance(short_df, long_df):
    
    predictors = long_df['distance(km)']
    short_df.loc[:, 'distance(km)'] = 0
    short_df.loc[:, 'flight_duration'] = 'NO DATA'

    # Generate indices for the DataFrame and the list
    df_indices = np.linspace(0, len(predictors) - 1, len(short_df))

    # Interpolate values from the list to match the length of the DataFrame
    interpolated_values = np.interp(df_indices, np.arange(len(predictors)), predictors)

    # Assign the interpolated values to the 'distance (km)' column
    short_df.loc[:, 'distance(km)'] = interpolated_values.round(2)

    df = pd.concat([short_df, long_df], axis=0)

    return df
    #print(short_df)
    

In [87]:
def cleanup(df):

    #clean the data
    df['flight_duration'] = df['flight_duration'].str.replace('+', '')

    #remove corrupted cases
    df['flight_duration'] = pd.to_timedelta(df['flight_duration'])
    threshold_duration = pd.Timedelta(hours=4, minutes=30)
    df = df[df['flight_duration'] <= threshold_duration]
    
    #predict the corrupted cases
    #short_df = df[df['distance(km)'] < 125]
    #long_df = df[df['distance(km)'] >= 125]

    #df = predict_distance(short_df, long_df)

    return df
    #display(short_df)
    #print(long_df))

    

In [88]:
def run_main():

    print("process started...")
    df = pd.read_csv(file_name)
    #print(df['distance(km)'])
    #df = df[df['distance(km)'] > min_threashold] #clean the corrupted flights
    df = cleanup(df)
    #print(df['distance(km)'])
    #df.drop(columns=['Unnamed: 0'], inplace = True)
    #print(df.columns)
    #print(df)
    #display(df)
    #df = df.iloc[2000:]
    #print(df)
    #df.fillna("NO DATA", inplace=True)
    #print(df)
    
    
    #df['src_airport'] = ""
    #df['src_country'] = ""
    #df['dest_airport'] = ""
    #df['dest_country'] = ""
    
    for index, row in df.iterrows():
        #print('for entered')
        src_lon = row['src_lon']
        src_lat = row['src_lat']
        dest_lon = row['dest_lon']
        dest_lat = row['dest_lat']

        if row['src_airport'] == 'NO DATA'or row['src_country'] == 'NO DATA':
            print('entered')
            time.sleep(req_window)
            src_airport, src_country = find_src_airport_and_country(src_lon, src_lat)
            df.at[index, 'src_airport'] = src_airport
            df.at[index, 'src_country'] = src_country
        if row['dest_airport'] == 'NO DATA' or row['dest_country'] == 'NO DATA':
            print('entered')
            time.sleep(req_window)
            dest_airport, dest_country = find_dest_airport_and_country(dest_lon, dest_lat)
            df.at[index, 'dest_airport'] = dest_airport
            df.at[index, 'dest_country'] = dest_country

    df.fillna("NO DATA", inplace=True)
    
    #separate data into smaller pieces for the api
    #chunk_size = 400
    #chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

    #results = []
    #for chunk in chunks:
        #processed_chunk = get_fuel_consumption(chunk)
        #results.append(processed_chunk)

    #df = pd.concat(results, ignore_index=True)
    
    
    #df.to_csv("database_complete.csv", index=False) 

    pretty_df_func(df)

    #print(df)
    
    print("process finished!")

In [89]:
import requests

url = "https://aviation-reference-data.p.rapidapi.com/airports/search"
    
querystring = {"lat": f"50.1098","lon": f"14.2619","radius":"10"}

headers = {
    "X-RapidAPI-Key": "1daecde3ebmshc01fb168d310d25p1e0b29jsn22e5a1c9059e",
    "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
}
    
response = requests.get(url, headers=headers, params=querystring)
print(response.json())

[{'iataCode': 'PRG', 'icaoCode': 'LKPR', 'name': 'Prague', 'alpha2countryCode': 'CZ', 'latitude': 50.1008, 'longitude': 14.26}]


In [93]:
def combine_two_halves():
    df_first = pd.read_csv('completed_first_half.csv')
    df_first.drop(columns='Unnamed: 0', inplace=True)
    df_second = pd.read_csv('completed_second_half.csv')

    merged_df = pd.concat([df_first, df_second], ignore_index=True)
    merged_df.drop(columns='Unnamed: 0', inplace=True)
    print(merged_df.columns)
    #display(merged_df)

    columns_to_keep = ['icao', 'distance(km)', 'model', 'co2', 'fuel', 'distance(km)', 'take_off_time', 'landing_time', 'flight_duration', 'src_country', 'src_airport', 'dest_country', 'dest_airport']
    merged_df = merged_df[columns_to_keep]
    print(len(merged_df))
    merged_df = merged_df[merged_df['icao'].shift(1) != merged_df['icao']]
    print(len(merged_df))
    merged_df.to_csv('completeddd.csv')

combine_two_halves()

Index(['icao', 'origin_country', 'lon', 'lat', 'alt', 'on_ground',
       'velocity(m/s)', 'true_track', 'vertical-rate', 'velocity(km/h)',
       'last_contact', 'distance(km)', 'src_lat', 'src_lon', 'take_off_time',
       'landing_time', 'took_off', 'landed', 'dest_lat', 'dest_lon',
       'flight_duration', 'src_airport', 'src_country', 'dest_airport',
       'dest_country', 'model', 'co2', 'fuel'],
      dtype='object')
3360
3247


In [ ]:
run_main()